In [116]:
import tensorflow as tf
import os
import json
from transformers import TFBertModel
answer_types=5
max_answer_length =30
model_dir = "/Users/aashnabanerjee/Documents/Cortx/inference/code/bert-joint-baseline"
MODEL_NAME = "bert-large-uncased-whole-word-masking-finetuned-squad"
train_filename = os.path.join(model_dir, "nq-train.tfrecords-00000-of-00001")
val_filename = os.path.join(model_dir, "eval.tf_record")
# val_filename2 = os.path.join(model_dir, "op.tf_record")

In [92]:
def get_dataset(tf_record_file, seq_length, batch_size=1, shuffle_buffer_size=0, is_training=False):

    if is_training:
        features = {
            "unique_ids": tf.io.FixedLenFeature([], tf.int64),
            "input_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
            "input_mask": tf.io.FixedLenFeature([seq_length], tf.int64),
            "segment_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
            "start_positions": tf.io.FixedLenFeature([], tf.int64),
            "end_positions": tf.io.FixedLenFeature([], tf.int64),
            "answer_types": tf.io.FixedLenFeature([], tf.int64)
        }
    else:
        features = {
            "unique_ids": tf.io.FixedLenFeature([], tf.int64),
            "input_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
            "input_mask": tf.io.FixedLenFeature([seq_length], tf.int64),
            "segment_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
            "token_map": tf.io.FixedLenFeature([seq_length], tf.int64)
        }        
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)
#         print(example)
        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
                        
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example

    def select_data_from_record(record):
        
        x = {
            'unique_ids': record['unique_ids'],
            'input_ids': record['input_ids'],
            'input_mask': record['input_mask'],
            'segment_ids': record['segment_ids']
        }
        if not is_training:
            x['token_map'] = record['token_map']

        if is_training:
            y = {
                'start_positions': record['start_positions'],
                'end_positions': record['end_positions'],
                'answer_types': record['answer_types']
            }

            return (x, y)
        
        return x

    dataset = tf.data.TFRecordDataset(tf_record_file)
    
    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    
    if shuffle_buffer_size > 0:
        dataset = dataset.shuffle(shuffle_buffer_size)
    
    dataset = dataset.batch(batch_size)
    
    return dataset

In [93]:
train_dataset = get_dataset(train_filename,
                    seq_length=512,
                    batch_size=2,
                    shuffle_buffer_size=10000,
                    is_training=True
                ) 

In [94]:
test_dataset = get_dataset(val_filename2,
                         seq_length=512,
                         batch_size=2,
                         is_training=False
                     )

In [95]:
while True:
    for x in test_dataset:
        print(x['unique_ids'])
# print(test_dataset)
# print(train_dataset)

tf.Tensor([-1679915555  -747773390], shape=(2,), dtype=int32)
tf.Tensor([ -914962702 -1679915548], shape=(2,), dtype=int32)
tf.Tensor([ -914962690 -2046721481], shape=(2,), dtype=int32)
tf.Tensor([ -413634991 -1679915562], shape=(2,), dtype=int32)
tf.Tensor([-735523475 -747773366], shape=(2,), dtype=int32)
tf.Tensor([-735523469 -413634970], shape=(2,), dtype=int32)
tf.Tensor([507926362 507926349], shape=(2,), dtype=int32)
tf.Tensor([-735523474 -747773358], shape=(2,), dtype=int32)
tf.Tensor([-413634979 -747773355], shape=(2,), dtype=int32)
tf.Tensor([-747773351 -747773360], shape=(2,), dtype=int32)
tf.Tensor([-413634973 -413634995], shape=(2,), dtype=int32)
tf.Tensor([-747773372 -633935952], shape=(2,), dtype=int32)
tf.Tensor([ -914962700 -2046721459], shape=(2,), dtype=int32)
tf.Tensor([ -735523464 -2046721466], shape=(2,), dtype=int32)
tf.Tensor([-1679915551  -735523497], shape=(2,), dtype=int32)
tf.Tensor([-413634983 -914962706], shape=(2,), dtype=int32)
tf.Tensor([ -747773354 -1679

tf.Tensor([-735523468 -413634987], shape=(2,), dtype=int32)
tf.Tensor([-747773345 -735523496], shape=(2,), dtype=int32)
tf.Tensor([-747773347 -413634986], shape=(2,), dtype=int32)
tf.Tensor([ -747773343 -2046721480], shape=(2,), dtype=int32)
tf.Tensor([ 507926352 -914962699], shape=(2,), dtype=int32)
tf.Tensor([-413634984 -413635000], shape=(2,), dtype=int32)
tf.Tensor([-735523509 -747773391], shape=(2,), dtype=int32)
tf.Tensor([1808428878 -735523488], shape=(2,), dtype=int32)
tf.Tensor([-2046721482  -735523500], shape=(2,), dtype=int32)
tf.Tensor([-735523476 -914962701], shape=(2,), dtype=int32)
tf.Tensor([-735523495 -735523503], shape=(2,), dtype=int32)
tf.Tensor([1808428879 -914962722], shape=(2,), dtype=int32)
tf.Tensor([-2046721477  -914962717], shape=(2,), dtype=int32)
tf.Tensor([-747773393 -747773361], shape=(2,), dtype=int32)
tf.Tensor([-2046721455   507926354], shape=(2,), dtype=int32)
tf.Tensor([507926356 507926350], shape=(2,), dtype=int32)
tf.Tensor([ -747773383 -2046721474

tf.Tensor([-413634973 -413634995], shape=(2,), dtype=int32)
tf.Tensor([-747773372 -633935952], shape=(2,), dtype=int32)
tf.Tensor([ -914962700 -2046721459], shape=(2,), dtype=int32)
tf.Tensor([ -735523464 -2046721466], shape=(2,), dtype=int32)
tf.Tensor([-1679915551  -735523497], shape=(2,), dtype=int32)
tf.Tensor([-413634983 -914962706], shape=(2,), dtype=int32)
tf.Tensor([ -747773354 -1679915566], shape=(2,), dtype=int32)
tf.Tensor([-1679915557  -747773374], shape=(2,), dtype=int32)
tf.Tensor([-2046721463  -914962711], shape=(2,), dtype=int32)
tf.Tensor([-914962685 -747773364], shape=(2,), dtype=int32)
tf.Tensor([-747773362 -633935957], shape=(2,), dtype=int32)
tf.Tensor([-747773356 -413634977], shape=(2,), dtype=int32)
tf.Tensor([ -914962686 -1679915563], shape=(2,), dtype=int32)
tf.Tensor([-914962712  507926360], shape=(2,), dtype=int32)
tf.Tensor([ -413634980 -2046721451], shape=(2,), dtype=int32)
tf.Tensor([-1679915554   507926351], shape=(2,), dtype=int32)
tf.Tensor([-413634992 

tf.Tensor([-1679915555  -747773390], shape=(2,), dtype=int32)
tf.Tensor([ -914962702 -1679915548], shape=(2,), dtype=int32)
tf.Tensor([ -914962690 -2046721481], shape=(2,), dtype=int32)
tf.Tensor([ -413634991 -1679915562], shape=(2,), dtype=int32)
tf.Tensor([-735523475 -747773366], shape=(2,), dtype=int32)
tf.Tensor([-735523469 -413634970], shape=(2,), dtype=int32)
tf.Tensor([507926362 507926349], shape=(2,), dtype=int32)
tf.Tensor([-735523474 -747773358], shape=(2,), dtype=int32)
tf.Tensor([-413634979 -747773355], shape=(2,), dtype=int32)
tf.Tensor([-747773351 -747773360], shape=(2,), dtype=int32)
tf.Tensor([-413634973 -413634995], shape=(2,), dtype=int32)
tf.Tensor([-747773372 -633935952], shape=(2,), dtype=int32)
tf.Tensor([ -914962700 -2046721459], shape=(2,), dtype=int32)
tf.Tensor([ -735523464 -2046721466], shape=(2,), dtype=int32)
tf.Tensor([-1679915551  -735523497], shape=(2,), dtype=int32)
tf.Tensor([-413634983 -914962706], shape=(2,), dtype=int32)
tf.Tensor([ -747773354 -1679

tf.Tensor([-735523482 -633935949], shape=(2,), dtype=int32)
tf.Tensor([ -914962723 -2046721483], shape=(2,), dtype=int32)
tf.Tensor([-735523481  507926358], shape=(2,), dtype=int32)
tf.Tensor([-735523498 1808428884], shape=(2,), dtype=int32)
tf.Tensor([-413634969 -735523483], shape=(2,), dtype=int32)
tf.Tensor([1808428877 -914962688], shape=(2,), dtype=int32)
tf.Tensor([-747773350], shape=(1,), dtype=int32)
tf.Tensor([-1679915555  -747773390], shape=(2,), dtype=int32)
tf.Tensor([ -914962702 -1679915548], shape=(2,), dtype=int32)
tf.Tensor([ -914962690 -2046721481], shape=(2,), dtype=int32)
tf.Tensor([ -413634991 -1679915562], shape=(2,), dtype=int32)
tf.Tensor([-735523475 -747773366], shape=(2,), dtype=int32)
tf.Tensor([-735523469 -413634970], shape=(2,), dtype=int32)
tf.Tensor([507926362 507926349], shape=(2,), dtype=int32)
tf.Tensor([-735523474 -747773358], shape=(2,), dtype=int32)
tf.Tensor([-413634979 -747773355], shape=(2,), dtype=int32)
tf.Tensor([-747773351 -747773360], shape=(2

...

tf.Tensor([-735523498 1808428884], shape=(2,), dtype=int32)
tf.Tensor([-413634969 -735523483], shape=(2,), dtype=int32)
tf.Tensor([1808428877 -914962688], shape=(2,), dtype=int32)
tf.Tensor([-747773350], shape=(1,), dtype=int32)
tf.Tensor([-1679915555  -747773390], shape=(2,), dtype=int32)
tf.Tensor([ -914962702 -1679915548], shape=(2,), dtype=int32)
tf.Tensor([ -914962690 -2046721481], shape=(2,), dtype=int32)
tf.Tensor([ -413634991 -1679915562], shape=(2,), dtype=int32)
tf.Tensor([-735523475 -747773366], shape=(2,), dtype=int32)
tf.Tensor([-735523469 -413634970], shape=(2,), dtype=int32)
tf.Tensor([507926362 507926349], shape=(2,), dtype=int32)
tf.Tensor([-735523474 -747773358], shape=(2,), dtype=int32)
tf.Tensor([-413634979 -747773355], shape=(2,), dtype=int32)
tf.Tensor([-747773351 -747773360], shape=(2,), dtype=int32)
tf.Tensor([-413634973 -413634995], shape=(2,), dtype=int32)
tf.Tensor([-747773372 -633935952], shape=(2,), dtype=int32)
tf.Tensor([ -914962700 -2046721459], shape=(2

tf.Tensor([-914962714 -735523510], shape=(2,), dtype=int32)
tf.Tensor([-413634994 -735523479], shape=(2,), dtype=int32)
tf.Tensor([ -735523477 -2046721472], shape=(2,), dtype=int32)
tf.Tensor([1808428883 -735523486], shape=(2,), dtype=int32)
tf.Tensor([-747773342 -747773348], shape=(2,), dtype=int32)
tf.Tensor([-2046721458  -914962698], shape=(2,), dtype=int32)
tf.Tensor([-1679915559 -2046721476], shape=(2,), dtype=int32)
tf.Tensor([-413634996 -747773395], shape=(2,), dtype=int32)
tf.Tensor([-2046721457 -1679915564], shape=(2,), dtype=int32)
tf.Tensor([-413634990 -747773363], shape=(2,), dtype=int32)
tf.Tensor([-747773385 -914962709], shape=(2,), dtype=int32)
tf.Tensor([-735523468 -413634987], shape=(2,), dtype=int32)
tf.Tensor([-747773345 -735523496], shape=(2,), dtype=int32)
tf.Tensor([-747773347 -413634986], shape=(2,), dtype=int32)
tf.Tensor([ -747773343 -2046721480], shape=(2,), dtype=int32)
tf.Tensor([ 507926352 -914962699], shape=(2,), dtype=int32)
tf.Tensor([-413634984 -4136350

tf.Tensor([-747773367 -747773341], shape=(2,), dtype=int32)
tf.Tensor([ -914962715 -2046721460], shape=(2,), dtype=int32)
tf.Tensor([  507926355 -1679915558], shape=(2,), dtype=int32)
tf.Tensor([-747773386 -914962693], shape=(2,), dtype=int32)
tf.Tensor([-914962716 -914962695], shape=(2,), dtype=int32)
tf.Tensor([1808428881 -914962713], shape=(2,), dtype=int32)
tf.Tensor([-747773394 -735523502], shape=(2,), dtype=int32)
tf.Tensor([ -747773388 -2046721461], shape=(2,), dtype=int32)
tf.Tensor([-914962694 -747773352], shape=(2,), dtype=int32)
tf.Tensor([-2046721475  -747773376], shape=(2,), dtype=int32)
tf.Tensor([ -735523465 -1679915568], shape=(2,), dtype=int32)
tf.Tensor([1808428882 -413634982], shape=(2,), dtype=int32)
tf.Tensor([-413634999 -747773392], shape=(2,), dtype=int32)
tf.Tensor([-1679915549  -914962720], shape=(2,), dtype=int32)
tf.Tensor([-735523508 -735523506], shape=(2,), dtype=int32)
tf.Tensor([-735523507 -735523492], shape=(2,), dtype=int32)
tf.Tensor([-413634993 -63393

tf.Tensor([-1679915555  -747773390], shape=(2,), dtype=int32)
tf.Tensor([ -914962702 -1679915548], shape=(2,), dtype=int32)
tf.Tensor([ -914962690 -2046721481], shape=(2,), dtype=int32)
tf.Tensor([ -413634991 -1679915562], shape=(2,), dtype=int32)
tf.Tensor([-735523475 -747773366], shape=(2,), dtype=int32)
tf.Tensor([-735523469 -413634970], shape=(2,), dtype=int32)
tf.Tensor([507926362 507926349], shape=(2,), dtype=int32)
tf.Tensor([-735523474 -747773358], shape=(2,), dtype=int32)
tf.Tensor([-413634979 -747773355], shape=(2,), dtype=int32)
tf.Tensor([-747773351 -747773360], shape=(2,), dtype=int32)
tf.Tensor([-413634973 -413634995], shape=(2,), dtype=int32)
tf.Tensor([-747773372 -633935952], shape=(2,), dtype=int32)
tf.Tensor([ -914962700 -2046721459], shape=(2,), dtype=int32)
tf.Tensor([ -735523464 -2046721466], shape=(2,), dtype=int32)
tf.Tensor([-1679915551  -735523497], shape=(2,), dtype=int32)
tf.Tensor([-413634983 -914962706], shape=(2,), dtype=int32)
tf.Tensor([ -747773354 -1679

KeyboardInterrupt: 

In [96]:
# Can't use next(train_dataset)!
while True:
    features, targets = next(iter(train_dataset))
    print(features['unique_ids'])


tf.Tensor([-1245041605  2144602347], shape=(2,), dtype=int32)
tf.Tensor([-2039605438  2051084877], shape=(2,), dtype=int32)
tf.Tensor([776432184 312150761], shape=(2,), dtype=int32)
tf.Tensor([1845779678 -846717197], shape=(2,), dtype=int32)
tf.Tensor([-1882811635 -1160375284], shape=(2,), dtype=int32)
tf.Tensor([-280761101 1606408004], shape=(2,), dtype=int32)


KeyboardInterrupt: 

In [97]:
class TFNQModel:
    
    def __init__(self, config, *inputs, **kwargs):
        """
        
        Subclasses of this class are different in self.backend,
        which should be a model that outputs a tensor of shape (batch_size, hidden_dim), and the
        `backend_call()` method.
        
        We will use Hugging Face Bert/DistilBert as backend in this notebook.
        """

        self.backend = None
        
        self.seq_output_dropout = tf.keras.layers.Dropout(kwargs.get('seq_output_dropout_prob', 0.05))
        self.pooled_output_dropout = tf.keras.layers.Dropout(kwargs.get('pooled_output_dropout_prob', 0.05))
        
        self.pos_classifier = tf.keras.layers.Dense(2,
                                        kernel_initializer=get_initializer(config.initializer_range),
                                        name='pos_classifier')  
              

        self.answer_type_classifier = tf.keras.layers.Dense(answer_types,
                                        kernel_initializer=get_initializer(config.initializer_range),
                                        name='answer_type_classifier')         
                
    def backend_call(self, inputs, **kwargs):
        """This method should be implemented by subclasses.
           
           The implementation should take into account the (somehow) different input formats of Hugging Face's
           models.
           
           For example, the `TFDistilBert` model, unlike `Bert` model, doesn't have segment_id as input.
           
           Then it calls `self.backend_call()` to get the outputs from Bert's model, which is used in self.call().
        """
        raise NotImplementedError

    
    def call(self, inputs, **kwargs):
        
        # sequence / [CLS] outputs from original bert
        sequence_output, pooled_output = self.backend_call(inputs, **kwargs)  # shape = (batch_size, seq_len, hidden_dim) / (batch_size, hidden_dim)
        
        # dropout
        sequence_output = self.seq_output_dropout(sequence_output, training=kwargs.get('training', False))
        pooled_output = self.pooled_output_dropout(pooled_output, training=kwargs.get('training', False))
        
        pos_logits = self.pos_classifier(sequence_output)  # shape = (batch_size, seq_len, 2)
        start_pos_logits = pos_logits[:, :, 0]  # shape = (batch_size, seq_len)
        end_pos_logits = pos_logits[:, :, 1]  # shape = (batch_size, seq_len)
        
        answer_type_logits = self.answer_type_classifier(pooled_output)  # shape = (batch_size, NB_ANSWER_TYPES)
        outputs = (start_pos_logits, end_pos_logits, answer_type_logits)

        return outputs  # logits
    
    
class TFBertForNQ(TFNQModel, TFBertPreTrainedModel):
    
    def __init__(self, config, *inputs, **kwargs):
        
        TFBertPreTrainedModel.__init__(self, config, *inputs, **kwargs)  # explicit calls without super
        TFNQModel.__init__(self, config)

        self.bert = TFBertMainLayer(config, name='bert')
        
    def backend_call(self, inputs, **kwargs):
        
        outputs = self.bert(inputs, **kwargs)
        sequence_output, pooled_output = outputs[0], outputs[1]  # shape = (batch_size, seq_len, hidden_dim) / (batch_size, hidden_dim)
        
        return sequence_output, pooled_output

model_mapping = {
    "bert": TFBertForNQ,
}


def get_pretrained_model(model_name):
    
    
    tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
    
    model_type = model_name.split("-")[0]
    if model_type not in model_mapping:
        raise ValueError("Model definition not found.")
    
    model_class = model_mapping[model_type]
    model = model_class.from_pretrained(MODEL_NAME)
    
    return tokenizer, model

bert_tokenizer, bert_for_nq = get_pretrained_model(MODEL_NAME)

In [98]:
# from transformers import TFBertMainLayer, TFBertPreTrainedModel, BertTokenizer
# from transformers.modeling_tf_utils import get_initializer

# class TFNQModel(TFBertPreTrainedModel):
    
#     def __init__(self, config, *inputs, **kwargs):
        
#         TFBertPreTrainedModel.__init__(self, config, *inputs, **kwargs)  # explicit calls without super
#         self.bert = TFBertMainLayer(config, name='bert')
#         self.backend = None
        
#         self.seq_output_dropout = tf.keras.layers.Dropout(kwargs.get('seq_output_dropout_prob', 0.05))
#         self.pooled_output_dropout = tf.keras.layers.Dropout(kwargs.get('pooled_output_dropout_prob', 0.05))
        
#         self.pos_classifier = tf.keras.layers.Dense(2,
#                                         kernel_initializer=get_initializer(config.initializer_range),
#                                         name='pos_classifier')       

#         self.answer_type_classifier = tf.keras.layers.Dense(answer_types,
#                                         kernel_initializer=get_initializer(config.initializer_range),
#                                         name='answer_type_classifier')         
                

#     def call(self, inputs, **kwargs):
        
#         # sequence / [CLS] outputs from original bert
#         outputs = self.bert(inputs, **kwargs)
#         sequence_output, pooled_output = outputs[0], outputs[1] 
        
#         # dropout
#         sequence_output = self.seq_output_dropout(sequence_output, training=kwargs.get('training', False))
#         pooled_output = self.pooled_output_dropout(pooled_output, training=kwargs.get('training', False))
        
#         pos_logits = self.pos_classifier(sequence_output)  # shape = (batch_size, seq_len, 2)
#         start_pos_logits = pos_logits[:, :, 0]  # shape = (batch_size, seq_len)
#         end_pos_logits = pos_logits[:, :, 1]  # shape = (batch_size, seq_len)
        
#         answer_type_logits = self.answer_type_classifier(pooled_output)  # shape = (batch_size, NB_ANSWER_TYPES)

#         outputs = (start_pos_logits, end_pos_logits, answer_type_logits)

#         return outputs
        
    
# def get_pretrained_model(MODEL_NAME):
    
#     pretrained_path = os.path.join(model_dir, MODEL_NAME)
    
#     tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
#     model = TFNQModel.from_pretrained(MODEL_NAME)
    
#     return tokenizer, model
# bert_tokenizer, bert_nq = get_pretrained_model(MODEL_NAME)

In [99]:
input_ids = tf.constant(bert_tokenizer.encode("Hello, my dog is cute"))[None, :]  # Batch size 1
input_masks = tf.constant(0, shape=input_ids.shape)
segment_ids = tf.constant(0, shape=input_ids.shape)

# Actual inputs to model
inputs = (input_ids, input_masks, segment_ids)
print(inputs[2])
print(input_ids.shape)
print(nq_inputs[2])

# Outputs from bert_for_nq using backend_call()
outputs = bert_nq(inputs)
(start_pos_logits, end_pos_logits, answer_type_logits) = outputs
print(start_pos_logits.shape)
print(end_pos_logits.shape)
print(answer_type_logits.shape)

len(bert_nq.trainable_variables)

tf.Tensor([[0 0 0 0 0 0 0 0]], shape=(1, 8), dtype=int32)
(1, 8)


NameError: name 'nq_inputs' is not defined

In [100]:
checkpoint_path = os.path.join(model_dir+"checkpoints", MODEL_NAME)
ckpt = tf.train.Checkpoint(model=bert_nq)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    last_epoch = int(ckpt_manager.latest_checkpoint.split("-")[-1])
    print (f'Latest BertNQ checkpoint restored -- Model trained for {last_epoch} epochs')
else:
    print('Checkpoint not found. Train BertNQ from scratch')
    last_epoch = 0
    

Checkpoint not found. Train BertNQ from scratch


In [152]:
import collections
n_best_size=20
max_answer_length = 30
Span = collections.namedtuple("Span", ["start_token_idx", "end_token_idx"])


class EvalExample(object):
    """Eval data available for a single example."""

    def __init__(self, example_id, candidates):
        self.example_id = example_id
        self.candidates = candidates
        self.results = {}
        self.features = {}


class ScoreSummary(object):

    def __init__(self):
        self.predicted_label = None
        self.short_span_score = None
        self.cls_token_score = None
        self.answer_type_logits = None
        self.start_prob = None
        self.end_prob = None
        self.answer_type_prob_dist = None

        
def read_candidates_from_one_split(input_path):
    """Read candidates from a single jsonl file."""
    candidates_dict = {}
    if input_path.endswith(".gz"):
        with gzip.GzipFile(fileobj=tf.io.gfile.GFile(input_path, "rb")) as input_file:
            print("Reading examples from: {}".format(input_path))
            for index, line in enumerate(input_file):
                e = json.loads(line)
                candidates_dict[e["example_id"]] = e["long_answer_candidates"]
                # if index > 100:
                #     break
    else:
        with tf.io.gfile.GFile(input_path, "r") as input_file:
            print("Reading examples from: {}".format(input_path))
            for index, line in enumerate(input_file):
                e = json.loads(line)
                candidates_dict[e["example_id"]] = e["long_answer_candidates"]
                # if index > 100:
                #     break


    return candidates_dict


def read_candidates(input_pattern):
    """Read candidates with real multiple processes."""
    input_paths = tf.io.gfile.glob(input_pattern)
    final_dict = {}
    for input_path in input_paths:
        final_dict.update(read_candidates_from_one_split(input_path))
    return final_dict

def get_best_indexes(logits, n_best_size, token_map=None):
    # Return a sorted list of (idx, logit)
    index_and_score = sorted(enumerate(logits[1:], 1), key=lambda x: x[1], reverse=True)
        
    best_indexes = []
    for i in range(len(index_and_score)):
        
        idx = index_and_score[i][0]

        if token_map is not None and token_map[idx] == -1:
            continue

        best_indexes.append(idx)

        if len(best_indexes) >= n_best_size:
            break    
    
    return best_indexes
def compute_predictions(example):
    """Converts an example into an NQEval object for evaluation.
    
       Unlike the starter kernel, this returns a list of `ScoreSummary`, sorted by score.
    """
    
    predictions = []

    for unique_id, result in example.results.items():
        
        if unique_id not in example.features:
            raise ValueError("No feature found with unique_id:", unique_id)
        token_map = example.features[unique_id]["token_map"].int64_list.value
        
        for start_index, start_logit, start_prob in zip(result["start_indexes"], result["start_logits"], result["start_pos_prob_dist"]):

            if token_map[start_index] == -1:
                continue            
            for end_index, end_logit, end_prob in zip(result["end_indexes"], result["end_logits"], result["end_pos_prob_dist"]):

                if token_map[end_index] == -1:
                    continue

                if end_index < start_index:
                    continue                    
                    
                length = end_index - start_index + 1
                if length > max_answer_length:
                    continue
                    
                summary = ScoreSummary()
                
                summary.instance_id = unique_id
                
                summary.short_span_score = start_logit + end_logit
                summary.cls_token_score = result["cls_start_logit"] + result["cls_end_logit"]
                summary.answer_type_logits = result["answer_type_logits"]
                
                summary.start_indexes = result["start_indexes"]
                summary.end_indexes = result["end_indexes"]

                summary.start_logits = result["start_logits"]
                summary.end_logits = result["end_logits"]                
                
                summary.start_pos_prob_dist = result["start_pos_prob_dist"]
                summary.end_pos_prob_dist = result["end_pos_prob_dist"]                
                           
                summary.start_index = start_index
                summary.end_index = end_index
                
                summary.start_logit = start_logit
                summary.end_logit = end_logit
                
                answer_type_prob_dist = result["answer_type_prob_dist"]
                summary.start_prob = start_prob
                summary.end_prob = end_prob
                summary.answer_type_prob_dist = {
                    "unknown": answer_type_prob_dist[0],
                    "yes": answer_type_prob_dist[1],
                    "no": answer_type_prob_dist[2],
                    "short": answer_type_prob_dist[3],
                    "long": answer_type_prob_dist[4]
                }
                start_span = token_map[start_index]
                end_span = token_map[end_index] + 1

                # Span logits minus the cls logits seems to be close to the best.
                score = summary.short_span_score - summary.cls_token_score
                predictions.append((score, summary, start_span, end_span))
                
    all_summaries = []            
                    
    if predictions:
        
        predictions = sorted(predictions, key=lambda x: (x[0], x[2], x[3]), reverse=True)
        
        for prediction in predictions:
            
            long_span = Span(-1, -1)
          
            score, summary, start_span, end_span = prediction
            short_span = Span(start_span, end_span)
            for c in example.candidates:
                start = short_span.start_token_idx
                end = short_span.end_token_idx
                if c["top_level"] and c["start_token"] <= start and c["end_token"] >= end:
                    long_span = Span(c["start_token"], c["end_token"])
                    break

            summary.predicted_label = {
                    "example_id": example.example_id,
                    "instance_id": summary.instance_id,
                    "long_answer": {
                            "start_token": long_span.start_token_idx,
                            "end_token": long_span.end_token_idx
                    },
                    "short_answers": [{
                            "start_token": short_span.start_token_idx,
                            "end_token": short_span.end_token_idx
                    }],
                    "yes_no_answer": "NONE",
                    "long_answer_score": score,                
                    "short_answers_score": score,                
                    "answer_type_prob_dist": summary.answer_type_prob_dist,
                    "start_index": summary.start_index,
                    "end_index": summary.end_index,
                    "start_logit": summary.start_logit,
                    "end_logit": summary.end_logit,
                    "start_prob": summary.start_prob,
                    "end_prob": summary.end_prob,
                    "start_indexes": summary.start_indexes,
                    "end_indexes": summary.end_indexes,
                    "start_logits": summary.start_logits,
                    "end_logits": summary.end_logits,
                    "start_pos_prob_dist": summary.start_pos_prob_dist,
                    "end_pos_prob_dist": summary.end_pos_prob_dist
            }
            
            all_summaries.append(summary)

    if len(all_summaries) == 0:

        short_span = Span(-1, -1)
        long_span = Span(-1, -1)
        score = 0
        summary = ScoreSummary()        
        
        summary.predicted_label = {
                "example_id": example.example_id,
                "instance_id": None,
                "long_answer": {
                        "start_token": long_span.start_token_idx,
                        "end_token": long_span.end_token_idx,
                        "start_byte": -1,
                        "end_byte": -1
                },
                "long_answer_score": score,
                "short_answers": [{
                        "start_token": short_span.start_token_idx,
                        "end_token": short_span.end_token_idx,
                        "start_byte": -1,
                        "end_byte": -1
                }],
                "short_answers_score": score,
                "yes_no_answer": "NONE"
        }        
        
        all_summaries.append(summary)
            
    all_summaries = all_summaries[:min(n_best_size, len(all_summaries))]        
    
    return all_summaries


def compute_pred_dict(candidates_dict, dev_features, raw_results):
    """Computes official answer key from raw logits.
    
       Unlike the starter kernel, each nq_pred_dict[example_id] is a list of `predicted_label`
       that is defined in `compute_predictions`.
    """

    raw_results_by_id = [(int(res["unique_id"]), 1, res, None) for res in raw_results]
    examples_by_id = [(int(tf.cast(int(k), dtype=tf.int32)), 0, v, k) for k, v in candidates_dict.items()]
    
    features_by_id = [(int(tf.cast(f.features.feature["unique_ids"].int64_list.value[0], dtype=tf.int32)), 2, f.features.feature, None) for f in dev_features]
    
    print('merging examples...')
    merged = sorted(examples_by_id + raw_results_by_id + features_by_id)
    print('done.')
    
    examples = []
    for idx, type_, datum, orig_example_id in merged:
        if type_ == 0: # Here, datum the list `long_answer_candidates`
            examples.append(EvalExample(orig_example_id, datum))
        elif type_ == 2: # Here, datum is a feature with `token_map`
            examples[-1].features[idx] = datum
        else: # Here, datum is a raw_result given by the model
            examples[-1].results[idx] = datum    
    
    # Construct prediction objects.
    summary_dict = {}
    nq_pred_dict = {}
    for e in examples:
        
        all_summaries = compute_predictions(e)
        summary_dict[e.example_id] = all_summaries
        nq_pred_dict[e.example_id] = [summary.predicted_label for summary in all_summaries]
        if len(nq_pred_dict) % 100 == 0:
            print("Examples processed: %d" % len(nq_pred_dict))

    return nq_pred_dict

In [157]:
import gzip
def get_prediction_json(mode, max_nb_pos_logits=-1):
    
    dataset = test_dataset
    predict_file = "/Users/aashnabanerjee/Documents/Cortx/inference/code/bert-joint-baseline/dev/tiny-dev/nq-dev-sample.no-annot.jsonl.gz"
    eval_features = validation_features
    prediction_output_file = "/Users/aashnabanerjee/Documents/Cortx/output.json"
    
    print(predict_file)
    print(prediction_output_file)
    
    all_results = []
    prediction_start_time = datetime.datetime.now()

    for (batch_idx, features) in enumerate(dataset):

        batch_start_time = datetime.datetime.now()

        unique_ids = features['unique_ids']
        token_maps = features['token_map']       
        
        (input_ids, input_masks, segment_ids) = (features['input_ids'], features['input_mask'], features['segment_ids'])
        
        nq_inputs = (input_ids, input_masks, segment_ids)
#         print(nq_inputs)
        nq_logits = bert_nq(nq_inputs,training=False)

        (start_pos_logits, end_pos_logits, answer_type_logits) = nq_logits

        unique_ids = unique_ids.numpy().tolist()
        
        token_maps = token_maps.numpy().tolist()
        
        start_pos_prob_dist = tf.nn.softmax(start_pos_logits, axis=-1).numpy().tolist()
        end_pos_prob_dist = tf.nn.softmax(end_pos_logits, axis=-1).numpy().tolist()
        answer_type_prob_dist = tf.nn.softmax(answer_type_logits, axis=-1).numpy().tolist()
        
        start_pos_logits = start_pos_logits.numpy().tolist()
        end_pos_logits = end_pos_logits.numpy().tolist()
        answer_type_logits = answer_type_logits.numpy().tolist()

        for uid, token_map, s, e, a, sp, ep, ap in zip(unique_ids, token_maps, start_pos_logits, end_pos_logits, answer_type_logits, start_pos_prob_dist, end_pos_prob_dist, answer_type_prob_dist):
            if max_nb_pos_logits < 0:
                max_nb_pos_logits = len(start_pos_logits)
            
            # full_start_logits = s
            # full_end_logits = e
            
            cls_start_logit = s[0]
            cls_end_logit = e[0]
        
            start_indexes = get_best_indexes(s, max_nb_pos_logits, token_map)
            end_indexes = get_best_indexes(e, max_nb_pos_logits, token_map)            
            
            s = [s[idx] for idx in start_indexes]
            e = [e[idx] for idx in end_indexes]
            sp = [sp[idx] for idx in start_indexes]
            ep = [ep[idx] for idx in end_indexes]            
        
            raw_result = {
                "unique_id": uid,
                "start_indexes": start_indexes,
                "end_indexes": end_indexes,
                "start_logits": s,
                "end_logits": e,
                "answer_type_logits": a,
                "start_pos_prob_dist": sp,
                "end_pos_prob_dist": ep,
                "answer_type_prob_dist": ap,
                "cls_start_logit": cls_start_logit,
                "cls_end_logit": cls_end_logit
            }
            all_results.append(raw_result)

        batch_end_time = datetime.datetime.now()
        batch_elapsed_time = (batch_end_time - batch_start_time).total_seconds()

        if (batch_idx + 1) % 1 == 0:
            print('Batch {} | Elapsed Time {}'.format(
                batch_idx + 1,
                batch_elapsed_time
            ))
            break
      
    prediction_end_time = datetime.datetime.now()
    prediction_elapsed_time = (prediction_end_time - prediction_start_time).total_seconds()

    print('\nTime taken for prediction: {} secs\n'.format(prediction_elapsed_time))
    print("-" * 80 + "\n")

    print("Going to candidates file")
    candidates_dict = read_candidates(predict_file)

    print ("setting up eval features")
    # eval_features = ...

    print ("compute_pred_dict")
    nq_pred_dict = compute_pred_dict(candidates_dict, eval_features, all_results)
#     print(nq_pred_dict)
    predictions_json = {"predictions": list(v[0] for v in nq_pred_dict.values())}

    print ("writing json")
    with tf.io.gfile.GFile(prediction_output_file, "w") as f:
        json.dump(predictions_json, f, indent=4)
        
    return predictions_json

def get_best_indexes(logits, n_best_size, token_map=None):
    # Return a sorted list of (idx, logit)
    index_and_score = sorted(enumerate(logits[1:], 1), key=lambda x: x[1], reverse=True)
        
    best_indexes = []
    for i in range(len(index_and_score)):
        
        idx = index_and_score[i][0]

        if token_map is not None and token_map[idx] == -1:
            continue

        best_indexes.append(idx)

        if len(best_indexes) >= n_best_size:
            break    
    
    return best_indexes

In [158]:
validation_features = (tf.train.Example.FromString(r.numpy()) for r in tf.data.TFRecordDataset(val_filename))
valid_predictions_json = get_prediction_json(mode='valid', max_nb_pos_logits=-1)

/Users/aashnabanerjee/Documents/Cortx/inference/code/bert-joint-baseline/dev/tiny-dev/nq-dev-sample.no-annot.jsonl.gz
/Users/aashnabanerjee/Documents/Cortx/output.json
Batch 1 | Elapsed Time 8.37361

Time taken for prediction: 8.4027 secs

--------------------------------------------------------------------------------

Going to candidates file
Reading examples from: /Users/aashnabanerjee/Documents/Cortx/inference/code/bert-joint-baseline/dev/tiny-dev/nq-dev-sample.no-annot.jsonl.gz
setting up eval features
compute_pred_dict
merging examples...
done.
Examples processed: 100
Examples processed: 200
writing json


In [159]:
!python -m nq_eval \
  --logtostderr \
  --gold_path= ./inference/code/bert-joint-baseline/dev/tiny-dev/nq-dev-sample.jsonl.gz \
  --predictions_path=./output.json

I0524 19:51:37.528847 4499570112 eval_utils.py:213] Reading predictions from file: ./output.json
Traceback (most recent call last):
  File "/Users/aashnabanerjee/opt/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/aashnabanerjee/opt/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/aashnabanerjee/Documents/Cortx/nq_eval.py", line 463, in <module>
    app.run(main)
  File "/Users/aashnabanerjee/opt/anaconda3/lib/python3.7/site-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/Users/aashnabanerjee/opt/anaconda3/lib/python3.7/site-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/Users/aashnabanerjee/Documents/Cortx/nq_eval.py", line 431, in main
    nq_pred_dict = util.read_prediction_json(FLAGS.predictions_path)
  File "/Users/aashnabanerjee/Documents/Cortx/eval_utils.py", line 221, in read_prediction_json
    long_span = S